In [57]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark session & context
spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

Import libraries for training the ML models

In [58]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression


When we read in Time Series data, we need to "bin" it appropriately.  

We need to decide what to do if we have several values in a bin.  Take the average?  Just pick one?

In [79]:
# units are seconds
ts_bin_size = 60 * 60 * 24  # Round to nearest day

Read the OHLC data

In [80]:
bitcoin_price_ohlc_DF = spark.read.csv("data/btc/price-ohlc.csv", inferSchema=True, header=True)
bitcoin_price_ohlc_DF = bitcoin_price_ohlc_DF.withColumn(
  'ts_bin',
  F.round(F.col('time') / ts_bin_size)
)
bitcoin_price_ohlc_DF.printSchema()


root
 |-- time: integer (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- ts_bin: double (nullable = true)



In [144]:
bitcoin_market_cap_DF = spark.read.csv("data/btc/market-cap.csv", inferSchema=True, header=True)
bitcoin_market_cap_DF = bitcoin_market_cap_DF.withColumn(
  'ts_bin',
  F.round(F.col('time') / ts_bin_size)
)
bitcoin_market_cap_DF.sort(F.desc('time')).limit(1).show()

+----------+--------------------+-------+
|      time|          market_cap| ts_bin|
+----------+--------------------+-------+
|1629331200|8.497396385194412E11|18858.0|
+----------+--------------------+-------+



In [124]:
bitcoin_transaction_count_DF = spark.read.csv("data/btc/transaction-count.csv", inferSchema=True, header=True)
bitcoin_transaction_count_DF = bitcoin_transaction_count_DF.withColumn(
  'ts_bin',
  F.round(F.col('time') / ts_bin_size)
)
bitcoin_transaction_count_DF.sort(F.desc('time')).limit(1).show()

+----------+-----------------+-------+
|      time|transaction_count| ts_bin|
+----------+-----------------+-------+
|1629331200|           253293|18858.0|
+----------+-----------------+-------+



Next, read in price average data

In [126]:
bitcoin_price_DF = spark.read.csv("data/btc/price.csv", inferSchema=True, header=True)
bitcoin_price_DF = bitcoin_price_DF.withColumn(
  'ts_bin',
  F.round(F.col('time') / ts_bin_size)
)
bitcoin_price_DF.show()


+----------+--------------------+-------+
|      time|               price| ts_bin|
+----------+--------------------+-------+
|1279324800|0.049510000000000005|14807.0|
|1279411200|             0.08584|14808.0|
|1279497600|              0.0808|14809.0|
|1279584000| 0.07473333333333332|14810.0|
|1279670400|             0.07921|14811.0|
|1279756800|            0.055945|14812.0|
|1279843200|0.062283333333333336|14813.0|
|1279929600|             0.05454|14814.0|
|1280016000|              0.0505|14815.0|
|1280102400|               0.056|14816.0|
|1280188800|0.059844444444444436|14817.0|
|1280275200|              0.0589|14818.0|
|1280361600| 0.06920000000000001|14819.0|
|1280448000| 0.06428333333333333|14820.0|
|1280534400|             0.06785|14821.0|
|1280620800|              0.0611|14822.0|
|1280707200|                0.06|14823.0|
|1280793600|                0.06|14824.0|
|1280880000|             0.05795|14825.0|
|1280966400|               0.061|14826.0|
+----------+--------------------+-

We can do a filtered binned join to join these together, which is necessary because our timestamps don't match perfectly

In [127]:
combined = bitcoin_price_DF.join(bitcoin_transaction_count_DF, bitcoin_price_DF.ts_bin == bitcoin_transaction_count_DF.ts_bin, 'outer') \
  .select(
    bitcoin_price_DF.ts_bin, 
    bitcoin_price_DF.time, 
    bitcoin_transaction_count_DF.transaction_count,
    bitcoin_price_DF.price
    ) \
  .sort(F.desc("time"))
combined.show()


+-------+----------+-----------------+------------------+
| ts_bin|      time|transaction_count|             price|
+-------+----------+-----------------+------------------+
|18859.0|1629417600|             null| 47151.67413602592|
|18858.0|1629331200|           253293|46717.578745203784|
|18857.0|1629244800|           247198| 44801.18871714413|
|18856.0|1629158400|           257594| 44695.35761921304|
|18855.0|1629072000|           263368|46004.484078602196|
|18854.0|1628985600|           186057| 47064.63037062539|
|18853.0|1628899200|           208910| 47096.94566198037|
|18852.0|1628812800|           260482| 47831.97604291269|
|18851.0|1628726400|           250810|44428.288466806436|
|18850.0|1628640000|           262290| 45593.63593070519|
|18849.0|1628553600|           260803| 45594.07627236636|
|18848.0|1628467200|           252508| 46284.97398221619|
|18847.0|1628380800|           191465| 43785.48181939978|
|18846.0|1628294400|           212062| 44689.86095932558|
|18845.0|16282

In [145]:
combined = combined.join(bitcoin_market_cap_DF, combined.ts_bin == bitcoin_market_cap_DF.ts_bin, 'outer') \
  .select(
    combined.ts_bin, 
    combined.time, 
    combined.transaction_count, 
    bitcoin_market_cap_DF.market_cap,
    combined.price
    ) \
  .sort(F.desc("time"))
combined.show()

+-------+----------+--------------------+------------------+
| ts_bin|      time|          market_cap|             price|
+-------+----------+--------------------+------------------+
|18859.0|1629417600|                null| 47151.67413602592|
|18858.0|1629331200|8.497396385194412E11|46717.578745203784|
|18857.0|1629244800|8.482791605982158E11| 44801.18871714413|
|18856.0|1629158400|8.638057886725898E11| 44695.35761921304|
|18855.0|1629072000|8.816383611551125E11|46004.484078602196|
|18854.0|1628985600|8.725546547980099E11| 47064.63037062539|
|18853.0|1628899200|8.850899196685634E11| 47096.94566198037|
|18852.0|1628812800|  8.6754351226538E11| 47831.97604291269|
|18851.0|1628726400|8.441146652576704E11|44428.288466806436|
|18850.0|1628640000| 8.65642112515162E11| 45593.63593070519|
|18849.0|1628553600|8.563157729837993E11| 45594.07627236636|
|18848.0|1628467200|8.429713176175574E11| 46284.97398221619|
|18847.0|1628380800|8.324156870518466E11| 43785.48181939978|
|18846.0|1628294400|8.17

In [146]:
feature_assembler = VectorAssembler(inputCols=["time", "market_cap", "transaction_count"], outputCol='VFeatures', handleInvalid='skip')
output = feature_assembler.transform(combined)
output.limit(2).show()

IllegalArgumentException: transaction_count does not exist. Available: ts_bin, time, market_cap, price

In [129]:
traindata, testdata = output.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='VFeatures', labelCol='price')
regressor = regressor.fit(traindata)

21/08/20 23:02:37 WARN Instrumentation: [284e18c9] regParam is zero, which might cause numerical instability and overfitting.


In [130]:
regressor.coefficients

DenseVector([0.0001, -0.0573])

In [131]:
pred = regressor.evaluate(testdata)

In [141]:
print("""
  Features Column: %s
  Label Column: %s
  Explained Variance: %s
  r Squared %s
  r Squared (adjusted) %s
""" % (
  pred.featuresCol,
  pred.labelCol,
  pred.explainedVariance,
  pred.r2,
  pred.r2adj
))

pred.r2
pred.r2adj

pred.predictions.show()



  Features Column: VFeatures
  Label Column: price
  Explained Variance: 58598254.485269256
  r Squared 0.4993382328797972
  r Squared (adjusted) 0.49829409989310225



+-------+----------+-----------------+------------------+--------------------+------------------+
| ts_bin|      time|transaction_count|             price|           VFeatures|        prediction|
+-------+----------+-----------------+------------------+--------------------+------------------+
|18838.0|1627603200|           255251| 42080.75491077187|[1.6276032E9,2552...| 22557.26864293567|
|18845.0|1628208000|           260651|  42826.8312632613|[1.628208E9,26065...| 22326.89480657049|
|18849.0|1628553600|           260803| 45594.07627236636|[1.6285536E9,2608...|22363.231978468888|
|18852.0|1628812800|           260482| 47831.97604291269|[1.6288128E9,2604...|22415.392198827525|
|18854.0|1628985600|           186057| 47064.63037062539|[1.6289856E9,1860...|26699.358127851418|
|18855.0|1629072000|           263368|46004.484078602196|[1.629072E9,26336...|22283.925191642367|
|18821.0|1626134400|           229780|32616.020911028223|[1.6261344E9,2297...| 23824.27187183895|
|18829.0|1626825600|